In [1]:
# Import dictionaries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import time
import json
import requests 
from datetime import timedelta
from config import api_key
from config import shared_secret
from scipy.stats import linregress

from stop_words import get_stop_words
stop_words = get_stop_words('english')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

import datetime
import csv
import os
import spotipy
from config import token, secret
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)


In [2]:
# Pull in lyric data
lyric_file = "billboard_lyrics_1964-2015.csv"
lyric_df = pd.read_csv(lyric_file, skipinitialspace=False, encoding="ISO-8859-1")
lyric_df

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0
5,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,1.0
6,7,help,the beatles,1965,help i need somebody help not just anybody hel...,3.0
7,8,cant you hear my heart beat,hermans hermits,1965,carterlewis every time i see you lookin my way...,5.0
8,9,crying in the chapel,elvis presley,1965,you saw me crying in the chapel the tears i s...,1.0
9,10,my girl,the temptations,1965,ive got sunshine on a cloudy day when its cold...,3.0


In [11]:
current_df = lyric_df.loc[lyric_df["Year"] == 2015,:]
current_df.head()


,Rank,Song,Artist,Year,Lyrics,Source
5000,1,uptown funk,mark ronson featuring bruno mars,2015,this hit that ice cold michelle pfeiffer that...,1.0
5001,2,thinking out loud,ed sheeran,2015,when your legs dont work like they used to be...,1.0
5002,3,see you again,wiz khalifa featuring charlie puth,2015,its been a long day without you my friend and...,1.0
5003,4,trap queen,fetty wap,2015,im like hey wassup hello seen yo pretty ass s...,1.0
5004,5,sugar,maroon 5,2015,im hurting baby im broken down i need your lo...,1.0


In [3]:
# Check field count
lyric_df.count()

Rank      5100
Song      5100
Artist    5100
Year      5100
Lyrics    4913
Source    4913
dtype: int64

In [5]:
# Mark empty spaces and remove blank ones
lyric_noblanks = lyric_df.replace("  ","NAN")
lyric_noblanks = lyric_noblanks.replace("","NAN")
lyric_noblanks = lyric_noblanks.dropna(how="any")
lyric_noblanks.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,NAN,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [6]:
# Check count again, knowing NAN is still present
lyric_noblanks.count()

Rank      4913
Song      4913
Artist    4913
Year      4913
Lyrics    4913
Source    4913
dtype: int64

In [9]:
# Variables for holding sentiments
compound_list = []
positive_list = []
neutral_list = []
negative_list = []

counter = 0

# Loop through lyrics
for row in lyric_noblanks["Lyrics"]:
    # Run Vader Analysis on 
    results = analyzer.polarity_scores(row)
    
    compound = results["compound"]
    pos = results["pos"]
    neu = results["neu"]
    neg = results["neg"]

    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)

    
    counter +=1
print(f'success: {counter}')
    

success: 4913


In [ ]:
lyric_add = lyric_noblanks
lyric_add["Sentiment"] = compound_list
lyric_add["Word Count"] = word_count_list
lyric_add["Repetition"] = word_repetition
lyric_add["Word Count"] = total_word_count
lyric_add["Mode Word"] = word_repeated

lyric_add.head()

In [ ]:
# Filter out songs with less than 50 words
lyric_filter = lyric_add.loc[lyric_add["Word Count"] > 50,:]

lyric_filter.to_csv("Output/lyric_filter.csv", index=False)

lyric_filter.head()

In [ ]:
# Analysis for which is more popular, positive or negative songs
# Positive is >0.75 and Negative is <-0.75
bins = [0, 1970, 1980, 1990, 2000, 2010, 2016]
group_names = ['<1970', '1970-1979', '1980-1989', '1990-1999', '2000-2009', '2010+']

sentiment_analysis_decade = lyric_filter.copy()
sentiment_analysis_decade["Decade"] = pd.cut(sentiment_analysis_decade["Year"], bins, labels=group_names)

pos_songs = []
neg_songs = []
tot_songs = []

for x in group_names:
    decade_count = sentiment_analysis_decade.loc[sentiment_analysis_decade["Decade"] == x]
    
    count_pos = decade_count.loc[decade_count["Sentiment"] > 0.75]
    pos_songs.append(len(count_pos["Sentiment"]))
    
    count_neg = decade_count.loc[decade_count["Sentiment"] < -0.75]
    neg_songs.append(len(count_neg["Sentiment"]))
    
    tot_songs.append(len(decade_count["Sentiment"]))

percent_pos = [a/b*100 for a,b in zip(pos_songs,tot_songs)]
percent_neg = [a/b*100 for a,b in zip(neg_songs,tot_songs)]
percent_neu = [100-a-b for a,b in zip(percent_pos,percent_neg)]

pos_analysis_df = pd.DataFrame({"Decade": group_names, "Percent Positive": percent_pos, "Percent Negative": percent_neg, "Percent Neutral": percent_neu})


pos_analysis_df = pos_analysis_df[["Decade","Percent Positive", "Percent Negative","Percent Neutral"]]

pos_analysis_table = pos_analysis_df.copy()
pos_analysis_table["Percent Positive"] = pos_analysis_table["Percent Positive"].map("{:.1f}%".format)
pos_analysis_table["Percent Negative"] = pos_analysis_table["Percent Negative"].map("{:.1f}%".format)
pos_analysis_table["Percent Neutral"] = pos_analysis_table["Percent Neutral"].map("{:.1f}%".format)


pos_analysis_table.to_excel("Output/positivity_analysis_table.xlsx", index=False)
pos_analysis_df

In [ ]:
x_axis=np.arange(6)
(slope, intercept, _, _, _) = linregress(np.arange(6), pos_analysis_df["Percent Positive"])
fit = slope * x_axis + intercept

pos_plot = pos_analysis_df[["Decade","Percent Positive"]]

fig, ax = plt.subplots()

ax.bar(x_axis,pos_plot["Percent Positive"], color="skyblue")
ax.plot(x_axis,fit, color="black")

plt.rcParams["figure.figsize"] = [9,5]
plt.title("Change in Positivity Over Time", fontsize=16, fontweight="bold")
plt.ylabel("Percent Positive")

plt.xticks(np.arange(6),group_names)
plt.ylim(0,75,5)

plt.savefig("Output/positivity_over_time.png")
print(pos_analysis_df)
plt.show()

In [ ]:
x_axis=np.arange(6)
(slope, intercept, _, _, _) = linregress(np.arange(6), pos_analysis_df["Percent Negative"])
fit = slope * x_axis + intercept

neg_plot = pos_analysis_df[["Decade","Percent Negative"]]

fig, ax = plt.subplots()

ax.bar(x_axis,neg_plot["Percent Negative"], color="red", alpha=0.75)
ax.plot(x_axis,fit, color="black")

plt.rcParams["figure.figsize"] = [9,5]

plt.title("Change in Negativity Over Time", fontsize=16, fontweight="bold")
plt.ylabel("Percent Negative")

plt.xticks(np.arange(6),group_names)
plt.ylim(0,30,5)

plt.savefig("Output/negativity_over_time.png")
plt.show()

In [ ]:
# Plot of Sentiment Analysis
plt.scatter(lyric_filter["Rank"], lyric_filter["Sentiment"], marker="x")
plt.rcParams["figure.figsize"] = [9,5]

plt.savefig("Output/vader_analysis_top_100.png")
plt.show()

In [ ]:
# Sentiment by decade

bins = [0, 1970, 1980, 1990, 2000, 2010, 2016]
group_names = ['<1970', '1970-1979', '1980-1989', '1990-1999', '2000-2009', '2010+']

sentiment_decade = lyric_filter.copy()
sentiment_decade["Decade"] = pd.cut(sentiment_decade["Year"], bins, labels=group_names)
sentiment_decade = sentiment_decade.groupby(["Decade"])
decade_average = sentiment_decade["Sentiment"].mean()

sentiment_decade = pd.DataFrame({"Average": decade_average})

sentiment_decade

In [ ]:
rep_ave = lyric_filter["Repetition"].mean()
rep_med = lyric_filter["Repetition"].median()
rep_max = lyric_filter["Repetition"].max()

rep_max_df = lyric_filter.loc[lyric_filter["Repetition"]==rep_max]
rep_max_lyrics = rep_max_df["Lyrics"]


sorted_rep = sorted(lyric_filter["Repetition"])
lq_index = len(sorted_rep) // 4
lq = sorted_rep[lq_index]
uq_index = lq_index*3
uq = sorted_rep[uq_index]
quartile_range = uq - lq


rep_df = pd.DataFrame({"Mean": rep_ave, "Median": rep_med, "Q1": lq, "Q3": uq, "Quartile Range": quartile_range}, index=[0])

rep_max_df

In [ ]:
plt.scatter(lyric_filter["Rank"], lyric_filter["Repetition"], marker="x")
plt.rcParams["figure.figsize"] = [9,5]

plt.title("Word Repetition in Songs", fontsize=16, fontweight="bold")
plt.xlabel("Rank")
plt.ylabel("Count of Word Repetition")

     
plt.show()

In [ ]:
# Word Repetition by Decade
bins = [0, 1970, 1980, 1990, 2000, 2010, 2016]
group_names = ['<1970', '1970-1979', '1980-1989', '1990-1999', '2000-2009', '2010+']

average_rep = []

rep_decade = lyric_filter.copy()
rep_decade["Decade"] = pd.cut(rep_decade["Year"], bins, labels=group_names)

for bin in group_names:
    filtered_df = rep_decade.loc[rep_decade["Decade"] == bin]
    average_rep.append(filtered_df["Repetition"].mean())
    
rep_per_decade = pd.DataFrame({"Decade": group_names, "Average Repetition": average_rep})
rep_per_decade = rep_per_decade[["Decade","Average Repetition"]]
rep_per_decade

In [ ]:
x_axis=np.arange(6)
(slope, intercept, _, _, _) = linregress(np.arange(6), rep_per_decade["Average Repetition"])
fit = slope * x_axis + intercept

fig, ax = plt.subplots()

ax.bar(x_axis,rep_per_decade["Average Repetition"], color="green", alpha=0.75)
ax.plot(x_axis,fit, color="black")

plt.rcParams["figure.figsize"] = [9,5]

plt.title("Change in Repetition Over Time", fontsize=16, fontweight="bold")
plt.ylabel("Repetition")

plt.xticks(np.arange(6),group_names)
plt.ylim(0,25,5)

plt.savefig("Output/word_repetition_over_time.png")
plt.show()

In [ ]:
# Add Song Length Component
song_length = 3.5

lyric_length = lyric_filter
lyric_length["Words Per Minute"] = lyric_length["Word Count"]/song_length

wpm_ave = lyric_length["Words Per Minute"].mean()
wpm_med = lyric_length["Words Per Minute"].median()

sorted_wpm = sorted(lyric_length["Words Per Minute"])
lq_index = len(sorted_wpm) // 4
lq = sorted_wpm[lq_index]
uq_index = lq_index*3
uq = sorted_wpm[uq_index]
quartile_range = uq - lq


wpm_df = pd.DataFrame({"Mean": wpm_ave, "Median": wpm_med, "Q1": lq, "Q3": uq, "Quartile Range": quartile_range}, index=[0])
wpm_df

In [ ]:
# Plot of words per minute
plt.scatter(lyric_length["Rank"], lyric_length["Word Count"], marker="x")
plt.rcParams["figure.figsize"] = [9,5]

plt.title("Song Word Count", fontsize=16, fontweight="bold")
plt.xlabel("Rank")
plt.ylabel("Words per Minute")

plt.savefig("Output/word_count_over_time.png")

plt.show()

In [ ]:
# Word Count by Decade
bins = [0, 1970, 1980, 1990, 2000, 2010, 2016]
group_names = ['<1970', '1970-1979', '1980-1989', '1990-1999', '2000-2009', '2010+']

wc_decade = lyric_filter.copy()
wc_decade["Decade"] = pd.cut(wc_decade["Year"], bins, labels=group_names)

average_wc = []

for bin in group_names:
    filtered_df = rep_decade.loc[wc_decade["Decade"] == bin]
    average_wc.append(filtered_df["Word Count"].mean())

wc_decade = pd.DataFrame({"Decade": group_names, "Average": average_wc})

wc_decade

In [ ]:
x_axis=np.arange(6)
(slope, intercept, _, _, _) = linregress(np.arange(6), wc_decade["Average"])
fit = slope * x_axis + intercept

fig, ax = plt.subplots()

ax.bar(x_axis,wc_decade["Average"], color="green", alpha=0.75)
ax.plot(x_axis,fit, color="black")

plt.rcParams["figure.figsize"] = [9,5]

plt.title("Change in Word Count Over Time", fontsize=16, fontweight="bold")
plt.ylabel("Word Count")

plt.xticks(np.arange(6),group_names)
#plt.ylim(0,25,5)

plt.savefig("Output/word_count_over_time.png")
plt.show()

##The below code was used to loop through the CSV and do the api call to get the BPM for each song. Due to the time it takes to loop through and make the call the code as been added as markdown. See Rachel_Master.ipynp for the code


#loop through and get beats per minute
from bpm_config import api_key
import requests
bpm = []
#tracks_per_year = lyric_noblanks.loc[lyric_noblanks["Year"]==2015]
for index, row in lyric_noblanks.iterrows():
    base_url = "https://api.getsongbpm.com/search/?"
    track = row["Song"]
    artist = row["Artist"]
    info = requests.get(f"{base_url}api_key={api_key}&type=both&lookup=song:{track} artist:{artist}").json()
    bpm.append(info)
print(bpm)

#Loop through and get the beats per minute otherwise use place holder of N/A
bpm_list = []


for track in bpm:
    try:
        bpm_list.append(track["search"][0]["tempo"])
    except (KeyError):
        bpm_list.append("N/A")

  
print(bpm_list)

#Create a series with the bpm list and add it as a column to the dataframe
bpm_series=pd.Series(bpm_list)
lyric_noblanks["BPM"] = bpm_series.values

lyric_filter = lyric_noblanks.loc[lyric_noblanks["BPM"] != "N/A"]
lyric_filter.head()

#Write to csv
lyric_noblanks.to_csv('BPM_CSV.csv')

In [ ]:
##End Markdown for BPM code

In [ ]:
#import csv and turn into dataframe
file_path = "BPM_CSV.csv"
bpm_df = pd.read_csv(file_path)
bpm_df.head()

In [ ]:
#Remove blank rows
clean_bpm_df = bpm_df.replace(" ","NaN")
clean_bpm_df = bpm_df.dropna(subset=["BPM"])
clean_bpm_df = clean_bpm_df.reset_index(drop=True)
clean_bpm_df = clean_bpm_df[["Rank","Song","Artist","Year","Lyrics", "Source", "BPM"]]
clean_bpm_df.head()


In [ ]:
#find average bpm
average_bpm=clean_bpm_df["BPM"].mean()
print(average_bpm)

#find min & max bpm
max_bpm=clean_bpm_df.loc[clean_bpm_df["BPM"].idxmax()]
min_bpm=clean_bpm_df.loc[clean_bpm_df["BPM"].idxmin()]
print(max_bpm)
print(min_bpm)

In [ ]:
#Scatterplot Year vs Beats per minute

plt.scatter(clean_bpm_df["Year"], clean_bpm_df["BPM"], color="gold")
plt.title('Beats per Minute vs Year')
plt.xlabel('Years')
plt.ylabel('Beats per Minute')
plt.xlim(1964,2016)
plt.hlines(120, 1964, 2016)
plt.savefig("Output/bpm_year.png")
plt.show()

In [ ]:
#Create Bins

bins = [0, 1970, 1980, 1990, 2000, 2010, 2016]
group_names = ['<1970', '1970-1979', '1980-1989', '1990-1999', '2000-2009', '2010+']
#new dataframe and groupby
decade_df = clean_bpm_df.copy()
decade_df["Decade"] = pd.cut(clean_bpm_df["Year"], bins, labels=group_names)
decade_groups = decade_df.groupby(["Decade"])

#find average bpm per decade
decade_average = decade_groups["BPM"].mean()
decade_count = decade_groups["BPM"].count()

bpm_decade_df = pd.DataFrame({
                      "BPM Average" : decade_average,})
                 

bpm_decade_df

In [ ]:
#Bar Chart of BPM and Decades
x_axis=np.arange(6)
(slope, intercept, _,_ , _) = stats.linregress(np.arange(6), bpm_decade_df["BPM Average"])
fit = slope * x_axis + intercept
plt.plot(fit, color="black")

plt.bar(x_axis, bpm_decade_df["BPM Average"], color="gold")

ax = plt.axes()
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.xticks(x_axis,group_names, rotation=45)

# Set a title for the chart
plt.ylabel("Beats Per Minute")
plt.title("Change in Beats Per Minute by Decade")
plt.savefig("Output/bpm_decade.png")
plt.show()


In [ ]:
#Create Scatter plot by chart ranking
plt.scatter(clean_bpm_df["Rank"], clean_bpm_df["BPM"], color="gold")
plt.title('Beats per Minute vs Chart Ranking')
plt.xlabel('Chart Rank')
plt.ylabel('Beats per Minute')
plt.xlim(0,101)
plt.hlines(120, 0, 101)
plt.savefig("Output/bpm_rank.png")
plt.show()

In [ ]:

#commenting out, so it's not run again#commen 
#Pulling in billboard data
#lyric_file = "billboard_lyrics_1964-2015.csv"
#lyric_df = pd.read_csv(lyric_file, skipinitialspace=False, encoding="ISO-8859-1")
#lyric_df.head()

In [ ]:

#commenting out, so it's not run again#commen 
#creating lists of songs from Song column
#song_list = lyric_df["Song"].tolist()
#song_list

In [ ]:
#commenting out, so it's not run again
#looking up spotify id's

#creating lists to store data
#song_name = []
#artist = []
#spotify_id = []

#looping through and pulling all songs that match the song name in the data set
#for id_lookup in song_list:
    #try:
#    time.sleep(10)

#    name = id_lookup

#    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
#    results1 = spotify.search(q='track:' + name, type='track')
#    results1
#    count = 0

#    for x in np.arange(len(results1["tracks"]["items"])):

#            spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
#            results = spotify.search(q='track:' + name, type='track')

#            song_name.append(results1["tracks"]["items"][count]["name"])

#            artist.append(results1["tracks"]["items"][count]["album"]["artists"][0]["name"])

#            spotify_id.append(results1["tracks"]["items"][count]["id"])
#            count += 1
        #except IndexError:
         #   next
#creating new dataframe
        #df['song name']  = song_name
        #df['artist']  = artist
        #df['id']  = spotify_id
#print(f'song {song_name} artist {artist} id {spotify_id}')


In [ ]:

#I took the dataframe and exported it, compared it to the original csv. Concatinated the artist and song names#I took 
#keeping only the records that matched the original data set.

In [ ]:
#Importing new dataset with Spotify ID's(so song details can be looked up)
file = 'billboard_lyrics_1964-2015-merge work.csv'
df = pd.read_csv(file, skipinitialspace=False, encoding="ISO-8859-1")
df.head()

In [ ]:
0]:
#creating lists to create new dataframe and to run through API calls
spotify_ids = df["spotify_ID"].tolist()
songnames = df["Song"].tolist()
artistnames = df["Artist"].tolist()
rank = df["Rank"].tolist()
year = df["Year"].tolist()

In [ ]:
#credentials needed for api call
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

#pulling only first 100 records to keep the api call from getting to many requests at once
x = 0
y = 99
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

#creating lists to store data retrieval
sng_nam = []
art_nam = []
rnk = []
yr = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
key = []
liveness = []
loudness = []
mode = []
speechiness = []
tempo = []
time_signature = []
valence = []

#creating counter for looping through records
cnt = 0
for id_lookup in id10:
    #credentials for api call
    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)

    #appending results into the lists created
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    #adding to the counter
    cnt += 1

In [ ]:
#running through results for the next set of 100

from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 100
y = 199
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 200
y = 299
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 300
y = 399
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 400
y = 499
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 500
y = 599
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 600
y = 699
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 700
y = 799
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
       
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 800
y = 899
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 900
y = 999
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1000
y = 1099
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1100
y = 1199
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1200
y = 1299
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1300
y = 1399
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1400
y = 1499
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1500
y = 1599
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1600
y = 1699
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1700
y = 1799
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
       
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1800
y = 1899
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 1900
y = 1999
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 2000
y = 2099
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
        
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 2100
y = 2199
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
     
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#running through results for the next set of 100
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=token, client_secret=secret)

x = 2200
y = 2226
id10 = spotify_ids[x:y]
idsong = songnames[x:y]
idartist = artistnames[x:y]
idrank = rank[x:y]
idyear = year[x:y]

cnt = 0
for id_lookup in id10:

    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    results = spotify.audio_features(id10)
    
    acousticness.append(results[cnt]["acousticness"])
    danceability.append(results[cnt]["danceability"])
    duration_ms.append(results[cnt]["duration_ms"])
    energy.append(results[cnt]["energy"])
    instrumentalness.append(results[cnt]["instrumentalness"])
    key.append(results[cnt]["key"])
    liveness.append(results[cnt]["liveness"])
    loudness.append(results[cnt]["loudness"])
    mode.append(results[cnt]["mode"])
    speechiness.append(results[cnt]["speechiness"])
    tempo.append(results[cnt]["tempo"])
    time_signature.append(results[cnt]["time_signature"])
    valence.append(results[cnt]["valence"])
    sng_nam.append(idsong[cnt])
    art_nam.append(idartist[cnt])
    rnk.append(idrank[cnt])
    yr.append(idyear[cnt])
    
    cnt += 1

In [ ]:
#CREATING DATA FRAME FROM SPOTIFY ID'S
df = pd.DataFrame()
df['song_name'] = sng_nam
df['rank'] = rnk
df['year'] = yr
df['artist_name'] = art_nam
df['acousticness']  = acousticness
df['danceability']  = danceability
df['duration_ms']  = duration_ms
df['energy']  = energy
df['instrumentalness']  = instrumentalness
df['key']  = key
df['liveness']  = liveness
df['loudness']  = loudness
df['mode']  = mode
df['speechiness']  = speechiness
df['tempo']  = tempo
df['time_signature']  = time_signature
df['valence']  = valence

df

In [ ]:
#converting the duration from milliseconds to minutes
df['duration_minutes']  = df['duration_ms']/60000

In [ ]:
#Scatterplot Year vs Acousticness
plt.scatter(df["year"], df["acousticness"], color = 'limegreen')
plt.title('Acousticness vs Year')
plt.xlabel('Years')
plt.ylabel('Acousticness')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Acousticness vs Year.png")
plt.show()

In [ ]:

#Scatterplot Danceability vs Year#Scatter 
plt.scatter(df["year"], df["danceability"], color = 'limegreen')
plt.title('Danceability vs Year')
plt.xlabel('Years')
plt.ylabel('Danceability')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Danceability vs Year.png")
plt.show()

In [ ]:
#Scatterplot Energy vs Year
plt.scatter(df["year"], df["energy"], color = 'limegreen')
plt.title('Energy vs Year')
plt.xlabel('Years')
plt.ylabel('Energy')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Energy vs Year.png")
plt.show()

In [ ]:
#Scatterplot Instrumentalness vs Year
plt.scatter(df["year"], df["instrumentalness"],color = 'limegreen')
plt.title('Instrumentalness vs Year')
plt.xlabel('Years')
plt.ylabel('Instrumentalness')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Instrumentalness vs Year.png")
plt.show()

In [ ]:
#Scatterplot Loudness vs Year
plt.scatter(df["year"], df["loudness"],color = 'limegreen')
plt.title('Loudness vs Year')
plt.xlabel('Years')
plt.ylabel('Loudness')
plt.xlim(1964,2016)
plt.ylim(-30,0)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Loudness vs Year.png")
plt.show()

In [ ]:
#Scatterplot Liveness vs Year
plt.scatter(df["year"], df["liveness"],color = 'limegreen')
plt.title('Liveness vs Year')
plt.xlabel('Years')
plt.ylabel('Liveness')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Liveness vs Year.png")
plt.show()

In [ ]:
#Scatterplot Speechiness vs Year
plt.scatter(df["year"], df["speechiness"],color = 'limegreen')
plt.title('Speechiness vs Year')
plt.xlabel('Years')
plt.ylabel('Speechiness')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Speechiness vs Year.png")
plt.show()

In [ ]:

#Scatterplot Tempo vs Year#Scatterp 
plt.scatter(df["year"], df["tempo"],color = 'limegreen')
plt.title('Tempo vs Year')
plt.xlabel('Years')
plt.ylabel('Tempo')
plt.xlim(1964,2016)
plt.ylim(40,225)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Tempo vs Year.png")
plt.show()

In [ ]:

#Scatterplot Valence vs Year#Scatterp 
plt.scatter(df["year"], df["valence"], color = 'limegreen')
plt.title('Valence vs Year')
plt.xlabel('Years')
plt.ylabel('Valence')
plt.xlim(1964,2016)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Valence vs Year.png")
plt.show()

In [ ]:
#Scatterplot Duration_minutes vs Year
plt.scatter(df["year"], df["duration_minutes"], color = 'limegreen')
plt.title('Duration_minutes vs Year')
plt.xlabel('Years')
plt.ylabel('Duration (In Minutes)')
plt.xlim(1964,2016)
plt.ylim(0,18)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Duration_minutes vs Year.png")
plt.show()

In [ ]:
#Scatter Rank vs Year
plt.scatter(df["rank"], df["acousticness"],color = 'orange')
plt.title('Acousticness vs Rank')
plt.xlabel('Rank')
plt.ylabel('Acousticness')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Acousticness vs Rank.png")
plt.show()

In [ ]:
#Scatter Danceability vs Rank
plt.scatter(df["rank"], df["danceability"], color="orange")
plt.title('Danceability vs Rank')
plt.xlabel('Rank')
plt.ylabel('Danceability')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Danceability vs Rank.png")
plt.show()

In [ ]:
#Scatter Duration_minutes vs rank
plt.scatter(df["rank"], df["duration_minutes"], color="orange")
plt.title('Duration_minutes vs rank')
plt.xlabel('Rank')
plt.ylabel('Durations in Minutes vs Rank')
plt.xlim(100,0)
plt.ylim(0,18)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Duration_minutes vs rank.png")
plt.show(

In [ ]:
#Scatter Energy vs Rank
plt.scatter(df["rank"], df["energy"], color = "orange")
plt.title('Energy vs Rank')
plt.xlabel('Rank')
plt.ylabel('Energy')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Energy vs Rank.png")
plt.show()

In [ ]:

#Scatter Instrumentalness vs Rank#Scatter  
plt.scatter(df["rank"], df["instrumentalness"], color = "orange")
plt.title('Instrumentalness vs Rank')
plt.xlabel('Rank')
plt.ylabel('Instrumentalness')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Instrumentalness vs Rank.png")
plt.show()

In [ ]:

#Scatter Speechiness vs Rank#Scatter  
plt.scatter(df["rank"], df["speechiness"],color = "orange")
plt.title('Speechiness vs Rank')
plt.xlabel('Rank')
plt.ylabel('Speechiness')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Speechiness vs Rank.png")
plt.show()

In [ ]:
#Scatter Liveness vs Rank
plt.scatter(df["rank"], df["liveness"],color = "orange")
plt.title('Liveness vs Rank')
plt.xlabel('Rank')
plt.ylabel('Liveness')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Liveness vs Rank.png")
plt.show()

In [ ]:
#Scatter Loudness vs Rank
plt.scatter(df["rank"], df["loudness"],color = "orange")
plt.title('Loudness vs Rank')
plt.xlabel('Rank')
plt.ylabel('Loudness')
plt.xlim(100,0)
plt.ylim(-30,0)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Loudness vs Rank.png")
plt.show()

In [ ]:

#Scatter Tempo vs Rank#Scatter  
plt.scatter(df["rank"], df["tempo"],color = "orange")
plt.title('Tempo vs Rank')
plt.xlabel('Rank')
plt.ylabel('Tempo')
plt.xlim(100,0)
plt.ylim(40,225)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Tempo vs Rank.png")
plt.show(

In [ ]:

#Scatter Valence vs Rank#Scatter  
plt.scatter(df["rank"], df["valence"],color = "orange")
plt.title('Valence vs Rank')
plt.xlabel('Rank')
plt.ylabel('Valence')
plt.xlim(100,0)
plt.ylim(0,1)
plt.grid()
plt.hlines(120, 1964, 2016)
plt.savefig("Output/Valence vs Rank.png")
plt.show()